<a href="https://colab.research.google.com/github/Nifdi01/object-hunter/blob/main/objecthunter/objecthunter/obd_models/yolov1/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementation of YOLOv1 Architecture in PyTorch

## Architecture Implementation
We will start by implementing the architecture of the YOLO mode from the [You Only Look Once](https://arxiv.org/abs/1506.02640) paper.

In [1]:
import torch
import torch.nn as nn

architecture_config = [
    # (kernel_size, input_channels, stride, padding)
    # "M" is max pool
    # This is a mix of Conv and MaxPool layers
    (7, 64, 2, 3),
    "M",
    (3, 192, 1, 1),
    "M",
    (1, 128, 1, 0),
    (3, 256, 1, 1),
    (1, 256, 1, 0),
    (3, 512, 1, 1),
    "M",
    # list with two conv_configs and the last digit (4) is the number of repititions of this layer
    [(1, 256, 1, 0), (3, 512, 1, 1), 4],
    (1, 512, 1, 0),
    (3, 1024, 1, 1),
    "M",
    [(1, 512, 1, 0), (3, 1024, 1, 1), 2],
    (3, 1024, 1, 1),
    (3, 1024, 2, 1),
    (3, 1024, 1, 1),
    (3, 1024, 1, 1)
]

In [2]:
class CNNBlock(nn.Module):
    """
    Convolutional Block that will serve as the building block for the YOLO Architecture.
    Inputs:
    in_channels: inputs
    out_channels: outputs

    Outputs:
    Forward propogation for leakyrelu(batchnorm(conv(x))) where x is our input
    """

    def __init__(self, in_channels, out_channels, **kwargs):
        super(CNNBlock, self).__init__()
        # Creates 2D Convolutional Layers
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        # Batch Normalization normalizes intermediate outputs; thus, making optimization faster and more stable
        self.batchnorm = nn.BatchNorm2d(out_channels)
        # As stated in the paper, we sill use Leaky ReLU with 0.1 slope
        self.leakyrelu = nn.LeakyReLU(0.1)

    def forward(self, x):
        return self.leakyrelu(self.batchnorm(self.conv(x)))

In [3]:
class YOLOv1(nn.Module):
    """
    Main engine class for YOLOv1. It will have everything needed for building the YOLOv1 model architecture.

    Inputs:
    in_channels: default is 3 in case we are using RGB images

    Outputs:
    Forward propogation for YOLOv1 architecture
    """

    def __init__(self, in_channels=3, **kwargs):
        super(YOLOv1, self).__init__()
        # We get the architecture configurations from the above
        self.architecture = architecture_config
        # We take the inputs
        self.in_channels = in_channels
        # The paper uses Darknet Framework by J. Redmon as the hidden layer architecture
        self.darknet = self.create_conv_layers(self.architecture)
        # The model should end with a fully-connected layer
        self.fully_connected_layer = self.create_fc_layer(**kwargs)

    def forward(self, x):
        x = self.darknet(x)
        # We flatten the Conv layer by dim=1 to flatten the number of classes
        return self.fully_connected_layer(torch.flatten(x, start_dim=1))

    # Implementation of darknet
    def create_conv_layers(self, architecture):
        # We will put layers into a list
        layers = []
        in_channels = self.in_channels

        # We traverse through the architecture
        for x in architecture:
            if type(x) == tuple:
                layers += [CNNBlock(in_channels, x[1],
                                    kernel_size=x[0],
                                    stride=x[2],
                                    padding=x[3])]
                in_channels=x[1]

            elif type(x) == str:
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]

            elif type(x) == list:
                conv1 = x[0]
                conv2 = x[1]
                loop_size = x[2]

                for _ in range(loop_size):
                    layers += [CNNBlock(in_channels,
                                        conv1[1],
                                        kernel_size=conv1[0],
                                        stride=conv1[2],
                                        padding=conv1[3])]

                    # The input for the following block is the output of the previous one. Therefore in_channels for the below layer is conv1[1]
                    layers += [CNNBlock(conv1[1],
                                        conv2[1],
                                        kernel_size=conv2[0],
                                        stride=conv2[2],
                                        padding=conv2[3])]
                    # Output of this conv2 layer is going to be input of the conv1 for the following repition
                    in_channels = conv2[1]

            # We unpack the list and convert it to nn.Sequential
            return nn.Sequential(*layers)

    # Implementation of Fully Connected Layer
    def create_fc_layer(self, split_size, num_boxes, num_classes):
        S, B, C = split_size, num_boxes, num_classes
        return nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * S * S, 496),
            nn.Dropout(0.0),
            nn.LeakyReLU(0.1),
            nn.Linear(496, S * S * (C + B * 5)),
        )

In [3]:
import torch
import torch.nn as nn

architecture_config = [
    # tuple = (kernel size, number of filters of output, stride, padding)
    (7, 64, 2, 3),
    "M",  # max-pooling 2x2 stride = 2
    (3, 192, 1, 1),
    "M",  # max-pooling 2x2 stride = 2
    (1, 128, 1, 0),
    (3, 256, 1, 1),
    (1, 256, 1, 0),
    (3, 512, 1, 1),
    "M",  # max-pooling 2x2 stride = 2
    # [tuple, tuple, repeat times]
    [(1, 256, 1, 0), (3, 512, 1, 1), 4],
    (1, 512, 1, 0),
    (3, 1024, 1, 1),
    "M",  # max-pooling 2x2 stride = 2
    # [tuple, tuple, repeat times]
    [(1, 512, 1, 0), (3, 1024, 1, 1), 2],
    (3, 1024, 1, 1),
    (3, 1024, 2, 1),
    (3, 1024, 1, 1),
    (3, 1024, 1, 1),
]


class CNNBlock(nn.Module):
    def __init__(self, in_channels, out_channels, **kwargs):
        super(CNNBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=False, **kwargs)
        self.batchnorm = nn.BatchNorm2d(out_channels)
        self.leakyrelu = nn.LeakyReLU(0.1)

    def forward(self, x):
        x = self.conv(x)
        x = self.batchnorm(x)
        x = self.leakyrelu(x)
        return x


class Yolov1(nn.Module):
    def __init__(self, in_channels=3, **kwargs):
        super(Yolov1, self).__init__()
        self.architecture = architecture_config
        self.in_channels = in_channels
        self.darknet = self._create_conv_layers(self.architecture)
        self.fcs = self._create_fcs(**kwargs)

    def forward(self, x):
        x = self.darknet(x)
        x = torch.flatten(x, start_dim=1)
        x = self.fcs(x)
        return x

    def _create_conv_layers(self, architecture):
        layers = []
        in_channels = self.in_channels

        for x in architecture:
            if type(x) == tuple:
                layers += [
                    CNNBlock(
                        in_channels, x[1], kernel_size=x[0], stride=x[2], padding=x[3],
                    )
                ]

                in_channels = x[1]

            elif type(x) == str:
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]

            elif type(x) == list:
                conv1 = x[0]  # tuple
                conv2 = x[1]  # tuple
                num_repeats = x[2]  # integer

                for _ in range(num_repeats):
                    layers += [
                        CNNBlock(
                            in_channels,
                            conv1[1],
                            kernel_size=conv1[0],
                            stride=conv1[2],
                            padding=conv1[3],
                        )
                    ]

                    layers += [
                        CNNBlock(
                            conv1[1],
                            conv2[1],
                            kernel_size=conv2[0],
                            stride=conv2[2],
                            padding=conv2[3],
                        )
                    ]

                    in_channels = conv2[1]

        return nn.Sequential(*layers)

    def _create_fcs(self, split_size, num_boxes, num_classes):
        S, B, C = split_size, num_boxes, num_classes
        return nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024*S*S, 4096),
            nn.Dropout(0.5),
            nn.LeakyReLU(0.1),
            nn.Linear(4096, S*S*(C+B*5)),  # (S,S,30)
        )


In [4]:
def test(S=7, B=2, C=20):
    model = Yolov1(split_size=S, num_boxes=B, num_classes=C)
    x = torch.randn((2, 3, 448, 448))
    print(model(x).shape)

test()

torch.Size([2, 1470])
